In [1]:
#%%
import os
import cv2
import numpy as np

#%%
imgSize = 128

def loadImage(directory):
    images, labels = [], []
    for label, folder in enumerate(['Melanoma', 'NotMelanoma']):
        path = os.path.join(directory, folder)
        for name in os.listdir(path):
            imgPath = os.path.join(path, name)
            img = cv2.imread(imgPath)
            if img is None:
                continue
            img = cv2.resize(img, (imgSize, imgSize))
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)
xTrain, yTrain = loadImage('train_sep')
xValid, yValid = loadImage('valid')
xTest, yTest = loadImage('test')

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(imgSize, imgSize, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


2024-12-02 20:53:33.826491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733190813.843171 1690061 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733190813.848245 1690061 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 20:53:33.867316: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/storage/icds/RISE/sw8/anaconda/conda_envs/tensorflow/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumP

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 4, 4, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,213,668 (16.07 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [3]:
history = model.fit(xTrain, yTrain, epochs=20, 
                    validation_data=(xValid, yValid), 
                    batch_size=16)


Epoch 1/20


I0000 00:00:1733190833.003083 1690440 service.cc:148] XLA service 0x147ea0002d00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733190833.003811 1690440 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-12GB, Compute Capability 6.0
2024-12-02 20:53:53.293461: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1733190834.825617 1690440 cuda_dnn.cc:529] Loaded cuDNN version 90300


 11/668 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.5373 - loss: 0.6854   

I0000 00:00:1733190842.725140 1690440 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


668/668 ━━━━━━━━━━━━━━━━━━━━ 43s 37ms/step - accuracy: 0.8380 - loss: 0.3598 - val_accuracy: 0.9130 - val_loss: 0.2244
Epoch 2/20
668/668 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.9038 - loss: 0.2354 - val_accuracy: 0.9175 - val_loss: 0.2059
Epoch 3/20
668/668 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.9133 - loss: 0.2114 - val_accuracy: 0.9194 - val_loss: 0.2000
Epoch 4/20
668/668 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9185 - loss: 0.1996 - val_accuracy: 0.9236 - val_loss: 0.1916
Epoch 5/20
668/668 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.9252 - loss: 0.1893 - val_accuracy: 0.9175 - val_loss: 0.1965
Epoch 6/20
668/668 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.9236 - loss: 0.1786 - val_accuracy: 0.9222 - val_loss: 0.1928
Epoch 7/20
668/668 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.9362 - loss: 0.1604 - val_accuracy: 0.9099 - val_loss: 0.2132
Epoch 8/20
668/668 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.9289 - loss: 0.1647 - val_accuracy: 0.9

In [7]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training & validation loss
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


AttributeError: module 'matplotlib' has no attribute 'get_data_path'

In [5]:
test_loss, test_acc = model.evaluate(xTest, yTest)
print(f"Test Accuracy: {test_acc:.2f}")


112/112 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - accuracy: 0.9461 - loss: 0.1360
Test Accuracy: 0.92


In [8]:
model.save('MelanomaModel.keras', save_format='keras_v3')

In [ ]:
import keras
import tensorflow as tf
print(tf.__version__)
print(keras.__version__)